# [XOR decryption](https://projecteuler.net/problem=59)

> Each character on a computer is assigned a unique code and the preferred standard is ASCII (American Standard Code for Information Interchange). For example, uppercase A = 65, asterisk (*) = 42, and lowercase k = 107.
>
> A modern encryption method is to take a text file, convert the bytes to ASCII, then XOR each byte with a given value, taken from a secret key. The advantage with the XOR function is that using the same encryption key on the cipher text, restores the plain text; for example, 65 XOR 42 = 107, then 107 XOR 42 = 65.
>
> For unbreakable encryption, the key is the same length as the plain text message, and the key is made up of random bytes. The user would keep the encrypted message and the encryption key in different locations, and without both "halves", it is impossible to decrypt the message.
>
> Unfortunately, this method is impractical for most users, so the modified method is to use a password as a key. If the password is shorter than the message, which is likely, the key is repeated cyclically throughout the message. The balance for this method is using a sufficiently long password key for security, but short enough to be memorable.
>
> Your task has been made easy, as the encryption key consists of three lower case characters. Using p059_cipher.txt (right click and 'Save Link/Target As...'), a file containing the encrypted ASCII codes, and the knowledge that the plain text must contain common English words, decrypt the message and find the sum of the ASCII values in the original text.


ASCII has been supplanted by UTF-8 as “the preferred standard” for character encoding and is what Julia uses, but UTF-8 was designed for backward compatibility with ASCII. As long as we are using characters with code points less than 128 (0x80), we don’t have to differentiate between the two standards.

In [1]:
using DelimitedFiles
C = readdlm("p059_cipher.txt", ',', UInt8) # vector of bytes
extrema(C)

(0x00, 0x5f)

We’re told that the encryption key is three characters long. We’ll reshape the cipher text so we can do frequency analysis on each row—data that was all encrypted with the same character.

In [2]:
keysize = 3
C′ = reshape(C, (keysize, cld(length(C), keysize)))

3×485 Array{UInt8,2}:
 0x24  0x00  0x17  0x11  0x04  0x0b  …  0x45  0x00  0x13  0x45  0x08  0x17
 0x16  0x00  0x19  0x58  0x13  0x58     0x19  0x58  0x1d  0x16  0x1a  0x0b
 0x50  0x04  0x13  0x04  0x15  0x16     0x02  0x15  0x1e  0x05  0x15  0x5e

Now, for each row, we’ll find the most frequent entry. We know the plain text is made up of common English words. Those will be separated by space characters (0x20), so this will be the most frequently occuring character in any reasonably long span of text. XORing the character code we identify in the cipher with 0x20 should recover the key character.

In [3]:
K = [] # key code points
for R in eachrow(C′)
    F = Dict(i => count(isequal(i), R) for i in unique(R)) # code point => occurences
    m = argmax(F) # most frequent code point in cipher
    k = m ⊻ 0x20 # the key for this row
    println(m => Char(k))
    push!(K, k)
end
K

0x45 => 'e'
0x58 => 'x'
0x50 => 'p'


3-element Array{Any,1}:
 0x65
 0x78
 0x70

It’s a good sign that all three results are lower-case letters, as hinted in the problem statement. Now that we have the letters of the key, ‘exp’, we can turn our attention to decrypting the cipher text. We’ll repeat the letters we found so that the key is the same length as the cipher text

In [4]:
Kᵣ = repeat(K, cld(length(C), length(K)))

1455-element Array{Any,1}:
 0x65
 0x78
 0x70
 0x65
 0x78
 0x70
 0x65
 0x78
 0x70
 0x65
 0x78
 0x70
 0x65
    ⋮
 0x65
 0x78
 0x70
 0x65
 0x78
 0x70
 0x65
 0x78
 0x70
 0x65
 0x78
 0x70

Now we can compute the bitwise XOR (Julia’s `⊻` operator—`\xor` or `\veebar`) for each element of the cipher and the key. Converting those code points back into characters reveals the plaintext. Adding them up gives the number required by the problem.

In [5]:
P = map(⊻, C, Kᵣ)
print(join(map(Char, P)))
Int(sum(P))

An extract taken from the introduction of one of Euler's most celebrated papers, "De summis serierum reciprocarum" [On the sums of series of reciprocals]: I have recently found, quite unexpectedly, an elegant expression for the entire sum of this series 1 + 1/4 + 1/9 + 1/16 + etc., which depends on the quadrature of the circle, so that if the true sum of this series is obtained, from it at once the quadrature of the circle follows. Namely, I have found that the sum of this series is a sixth part of the square of the perimeter of the circle whose diameter is 1; or by putting the sum of this series equal to s, it has the ratio sqrt(6) multiplied by s to 1 of the perimeter to the diameter. I will soon show that the sum of this series to be approximately 1.644934066842264364; and from multiplying this number by six, and then taking the square root, the number 3.141592653589793238 is indeed produced, which expresses the perimeter of a circle whose diameter is 1. Following again the same ste

129448